In [1]:
import findspark
findspark.init()

In [2]:
import json, requests, sys
from nltk.corpus import stopwords
from operator import add
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from textblob import TextBlob

In [3]:
# text classification
def sendTopWords(counts, url):
    def takeAndSend(time, rdd):
        if not rdd.isEmpty():
            word_counts = rdd.take(10)

            words = []
            values = []

            for (word, count) in word_counts:
                words.append(word)
                values.append(count)

            json_data = {'words': str(words), 'counts': str(values)}
            print(json_data)

            response = requests.post(url, data=json_data)

    counts.foreachRDD(takeAndSend)


def sendTweets(tweets, url):
    def takeAndSend(time, rdd):
        if not rdd.isEmpty():
            tweets_data = rdd.take(10)

            users = []
            texts = []
            tweet_ids = []

            for (user, text, follower_count, tweet_id) in tweets_data:
                users.append(user)
                texts.append(text)
                tweet_ids.append(tweet_id)

            json_data = {'user': str(users), 'text': str(texts), 'id': str(tweet_ids)}
            print(json_data)

            response = requests.post(url, data=json_data)

    tweets.foreachRDD(takeAndSend)


def sendTweetSentiments(sentiments, url):
    def takeAndSend(time, rdd):
        if not rdd.isEmpty():
            (name, (total, (pos, neutral, neg))) = rdd.first()

            json_data = {'positive': pos, 'neutral': neutral, 'negative': neg, 'total': total}
            print(json_data)

            response = requests.post(url, data=json_data)

    sentiments.foreachRDD(takeAndSend)


def sendGeoData(path, url):
    filepath = "file:///" + path
    geodata = sc.textFile(filepath) \
                .map(lambda x: x.encode("ascii", "ignore")) \
                .map(lambda x: json.loads(x)) \
                .map(lambda json_object: (json_object["user"]["screen_name"], json_object["coordinates"])) \
                .map(lambda kv: (kv[1]['coordinates'][0], kv[1]['coordinates'][1])) \
                .collect()

    longitudes = []
    latitudes = []

    for geotweet in geodata:
        longitudes.append(geotweet[0])
        latitudes.append(geotweet[1])

    json_data = {'longitude': str(longitudes), 'latitude': str(latitudes)}
    response = requests.post(url, data=json_data)


def sendTweetsFromStream(kvs, url):
    tweets = kvs.map(lambda x: json.loads(x)) \
                .map(lambda json_object: (json_object["user"]["screen_name"], json_object["text"], json_object["user"]["followers_count"], json_object["id"])) \
                .transform(lambda rdd: rdd.sortBy(lambda x: x[2], ascending = False))
    tweets.pprint()
    sendTweets(tweets, url)


def getSentiment(text):
    sent = TextBlob(text).sentiment.polarity

    if sent > 0:
        return (1, 0, 0)
    elif sent == 0:
        return (0, 1, 0)
    else:
        return (0, 0, 1)


def sendTweetSentimentsFromStream(kvs, url):
    sentiments = kvs.map(lambda x: json.loads(x)) \
                    .map(lambda json_object: (json_object["user"]["screen_name"], json_object["text"], getSentiment(json_object["text"]))) \
                    .map(lambda kv: ('count', (1, kv[2]))) \
                    .reduceByKey(lambda a, b: (a[0] + b[0], (a[1][0] + b[1][0], a[1][1] + b[1][1], a[1][2] + b[1][2])))
    sentiments.pprint()
    sendTweetSentiments(sentiments, url)


def sendTopHashtagsFromStream(kvs, url):
    tweets = kvs.map(lambda x: json.loads(x)) \
                .map(lambda json_object: (json_object["user"]["screen_name"], json_object["text"]))

    lines = tweets.flatMap(lambda line: line[1].split(" "))

    ## This part does the hashtag count
    hashtag_counts = lines.filter(lambda word: len(word) >= 2 and word[0] == '#') \
                          .map(lambda word: (word, 1)) \
                          .reduceByKey(add) \
                          .transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending = False))
    hashtag_counts.pprint()
    sendTopWords(hashtag_counts, url)


def sendTopWordsFromStream(kvs, url):
    tweets = kvs.map(lambda x: json.loads(x)) \
                .map(lambda json_object: (json_object["user"]["screen_name"], json_object["text"]))

    lines = tweets.flatMap(lambda line: line[1].split(" "))

    ## This part does the word count
    sw = stopwords.words('english')
    sw.extend(['rt', 'https', 'http', 'coronavirus', 'covid19', 'covid-19'])

    counts = lines.map(lambda word: word.strip().lower()) \
                  .filter(lambda word: word not in sw) \
                  .filter(lambda word: len(word) >= 2 and word[0] != '#' and word[0] != '@') \
                  .map(lambda word: (word, 1)) \
                  .reduceByKey(add) \
                  .transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending = False))
    counts.pprint()
    sendTopWords(counts, url)


In [4]:
sc = SparkContext(appName="tweetStream")
# Create a local StreamingContext with batch interval of 2 second
ssc = StreamingContext(sc, 2)
# Create a DStream that conencts to hostname:port
kvs = ssc.socketTextStream("0.0.0.0", 5555)

In [5]:
server = 'http://localhost:5000/'
geodata_path='/Users/shawvin/Desktop/Big data project/geo_tweets.txt'

sendGeoData(geodata_path, server + 'update_geodata')
sendTweetsFromStream(kvs, server + 'update_tweets')
sendTopHashtagsFromStream(kvs, server + 'update_hashtagcounts')
sendTopWordsFromStream(kvs, server + 'update_counts')
sendTweetSentimentsFromStream(kvs, server + 'update_sentiments')

In [ ]:
# Start computing
ssc.start()        
# Wait for termination
ssc.awaitTermination()
ssc.stop(stopGraceFully = True)

-------------------------------------------
Time: 2021-05-25 23:54:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-25 23:54:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-25 23:54:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-25 23:54:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-25 23:54:56
-------------------------------------------
('dianerocks52', 'RT @judgeyourself99: Two places Republicans don’t seem to have a problem with the election results. South Carolina and Kentucky. @LindseyGr…', 3469, 1397219613037969409)
('Barbara49962035', 'RT @Johnheretohelp: So many moving pieces! Nancy normally hides behind others giving the orders. Staying safe in her cocoon. But on January…', 2640, 1397219615084892165)
('ScrewyTrash', 'RT @Johnheretohelp: So many mo

-------------------------------------------
Time: 2021-05-25 23:55:00
-------------------------------------------
('#Kayah', 1)
('#Chin,', 1)

{'words': "['#Kayah', '#Chin,']", 'counts': '[1, 1]'}
-------------------------------------------
Time: 2021-05-25 23:55:00
-------------------------------------------
('presidential', 2)
('whole', 2)
('guns.', 2)
('republicans', 2)
('tatmadaw', 1)
('like', 1)
('honey', 1)
('potted', 1)
('destroying', 1)
('airtime', 1)
...

{'words': "['presidential', 'whole', 'guns.', 'republicans', 'tatmadaw', 'like', 'honey', 'potted', 'destroying', 'airtime']", 'counts': '[2, 2, 2, 2, 1, 1, 1, 1, 1, 1]'}
-------------------------------------------
Time: 2021-05-25 23:55:00
-------------------------------------------
('count', (5, (2, 2, 1)))

{'positive': 2, 'neutral': 2, 'negative': 1, 'total': 5}
-------------------------------------------
Time: 2021-05-25 23:55:02
-------------------------------------------
('kcingram50', "RT @MattBatzel: Wisconsin Electi

-------------------------------------------
Time: 2021-05-25 23:55:06
-------------------------------------------

-------------------------------------------
Time: 2021-05-25 23:55:06
-------------------------------------------
('perpetuate,', 1)
('me,', 1)
('designed', 1)
('perpetuate', 1)
('2020', 1)
('phone', 1)
('next', 1)
('insurrection,…', 1)
('vote', 1)
('sohai.', 1)
...

{'words': "['perpetuate,', 'me,', 'designed', 'perpetuate', '2020', 'phone', 'next', 'insurrection,…', 'vote', 'sohai.']", 'counts': '[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]'}
-------------------------------------------
Time: 2021-05-25 23:55:06
-------------------------------------------
('count', (3, (1, 2, 0)))

{'positive': 1, 'neutral': 2, 'negative': 0, 'total': 3}
-------------------------------------------
Time: 2021-05-25 23:55:08
-------------------------------------------
('1SnoozyQ', 'RT @Rosenlaw: Ipsos poll: 56% of Republicans believe the election was rigged or the result of illegal voting, and 53% think 

{'words': "['election', 'it’s', 'destroying', 'na', 'preserving', 'h.r.', 'integrity,', 'it.', 'yarn', 'perpetuate,']", 'counts': '[4, 3, 2, 2, 2, 2, 2, 2, 1, 1]'}
-------------------------------------------
Time: 2021-05-25 23:55:10
-------------------------------------------
('count', (8, (4, 1, 3)))

{'positive': 4, 'neutral': 1, 'negative': 3, 'total': 8}
-------------------------------------------
Time: 2021-05-25 23:55:12
-------------------------------------------
('GOP_Jack', 'RT @mikepompeo: H.R. 1 is not about preserving election integrity, it’s about destroying it.', 23608, 1397219680004235265)
('CaptainResists', 'RT @SenJeffMerkley: The powerful know that when the people come together, we demand things like lower health costs, better pay, and cleaner…', 11693, 1397219684081147913)
('weeser1', 'RT @bjwinnerdavis: What makes me angry is knowing that  #Manafort actively colluded with Russians to undermine our elections &amp; is free, par…', 11445, 1397219681338023943)
('Sullya

-------------------------------------------
Time: 2021-05-25 23:55:16
-------------------------------------------

-------------------------------------------
Time: 2021-05-25 23:55:16
-------------------------------------------
('election', 5)
('next', 2)
('vote', 2)
('sohai.', 2)
('presidential', 2)
('trump', 2)
('republicans', 2)
('sohai', 2)
('destroying', 1)
('backed', 1)
...

{'words': "['election', 'next', 'vote', 'sohai.', 'presidential', 'trump', 'republicans', 'sohai', 'destroying', 'backed']", 'counts': '[5, 2, 2, 2, 2, 2, 2, 2, 1, 1]'}
-------------------------------------------
Time: 2021-05-25 23:55:16
-------------------------------------------
('count', (8, (2, 4, 2)))

{'positive': 2, 'neutral': 4, 'negative': 2, 'total': 8}
-------------------------------------------
Time: 2021-05-25 23:55:18
-------------------------------------------
('EatlovePray11', 'Don’t forget , Durham has it all ! This will drop simultaneously with the Election Audits proving fraud ! Disclosur

...

-------------------------------------------
Time: 2021-05-26 00:08:18
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:18
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:20
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:20
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:20
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:20
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:22
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:22
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:22
----------

-------------------------------------------
Time: 2021-05-26 00:08:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:56
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:56
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:56
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:56
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:58
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:58
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:08:58
----------

-------------------------------------------
Time: 2021-05-26 00:09:30
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:30
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:32
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:32
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:32
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:32
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:34
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:34
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:09:34
----------

-------------------------------------------
Time: 2021-05-26 00:10:06
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:06
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:08
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:08
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:08
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:08
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:10
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:10
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:10
----------

-------------------------------------------
Time: 2021-05-26 00:10:42
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:42
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:44
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:44
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:44
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:44
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:46
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:46
-------------------------------------------

-------------------------------------------
Time: 2021-05-26 00:10:46
----------